In [7]:
import math
import random
from torchvision import datasets ,transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from torch import nn 
from torch.nn.functional import relu ,softmax 
import copy 
from torch.utils.data import Subset
import pandas as pd 
import numpy as np
from PIL import Image
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
device="cpu"

In [1]:
import pandas as pd
path ='Hand_Written_script_classification data/Line-level-Features/DHT_Algorithm_144_Features.csv'
#Write the path of the dataset
df = pd.read_csv(path) #header=None
df.head()

,att_1,att_2,att_3,att_4,att_5,att_6,att_7,att_8,att_9,att_10,...,att_136,att_137,att_138,att_139,att_140,att_141,att_142,att_143,att_144,class
0,279,60,39,36,24,21,20,19,20,22,...,3993,2419,3320,2660,2850,2591,2594,2444,2546,A
1,272,62,43,27,24,20,20,20,25,18,...,2687,2887,3494,2496,2591,2745,2384,2351,2411,A
2,177,82,46,28,23,22,19,16,17,16,...,3071,3717,3073,2835,3094,2499,2480,2483,2548,A
3,205,84,44,26,25,23,20,20,20,28,...,3818,2340,2372,2372,2370,2537,2526,2521,2418,A
4,166,91,38,26,25,21,19,19,19,17,...,2388,2561,2558,2543,3033,2928,2825,2530,2424,A


In [2]:
print(df.shape)

(3600, 145)


In [3]:
print(df.shape)
#X=df.loc[:,df.columns != 'Classifier'].values
X=df.loc[:,df.columns != df.columns[-1]].values
print(type(X))
print(X.shape)
#y_label=df['Classifier'].values 
y_label =df.iloc[:,-1].values
#print(type(y_label))
#print(X[0])
print()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_label)
print(le.classes_)
y=le.transform(y_label)
print(y[:10])

(3600, 145)
<class 'numpy.ndarray'>
(3600, 144)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L']
[0 0 0 0 0 0 0 0 0 0]


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler #for normalization
sc = StandardScaler()
X = sc.fit_transform(X)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0, 1))
# X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
# print(y_train)
# print(y_test)
#print(len(X_train), len(y_train), len(X_test), len(y_test))
# # X=X_train
# # y=y_train
print(set(y_train), set(y_test))
print(X.shape, y.shape)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11} {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
(3600, 144) (3600,)


In [8]:
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

In [9]:
#n_inputs = X_train.shape[1]
#n_hidden = 180
#n_classes = len(le.classes_)
#N = X_train.shape[0] # Number of samples
#print(N)
#hyperparameters

input_size = X_train.shape[1]    #The image size = 28 x 28 = 784
hidden_size = X_train.shape[1]    #The number of nodes at the hidden layer
num_classes = len(le.classes_)       #The number of output classes. In this case, from 0 to 9
num_epochs = 100      # The number of times entire dataset is trained
batch_size = X_train.shape[0]    # The size of input data took for one iteration
lr = 0.001  # The speed of convergence

In [10]:

#build model
class Model(nn.Module):
    def __init__(self,input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size , hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.drop = nn.Dropout (p=0.4)

    def masking (self,act1,mask,p):
        if (self.training ==True ):
            return ((act1*mask)/p)
        else :
            return (act1)
        

    def Forward(self, x,mask,p):
        x = F.relu(self.fc1(x))
        x = self.masking (x,mask,p)
        x = self.fc2(x)
        return x

In [11]:
#the mask has to be specified 
def train_model(model, criterion, optimizer,mask,p, num_epochs=25):
  
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    losses=[]
    accuracies=[]

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
              if phase == 'train':
                model.train()  # Set model to training mode
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs= model.Forward(X_train,mask,p) #inputs
                    train_loss = criterion(outputs, y_train)# backward + optimize only if in training phase,labels
                    train_loss.backward()
                    optimizer.step()
                train_acc = 100 * torch.sum(y_train==torch.max(outputs.data, 1)[1]).double() / len(y_train)
                print('********************{}*************'.format(phase))
                print ('Epoch [%d/%d] Loss: %.4f   Acc: %.4f'%(epoch+1, num_epochs, train_loss.item(), train_acc.item()))
                losses.append(train_loss)
                accuracies.append(train_acc)
            else:
                model.eval()   # Set model to evaluate mode
                outputs= model.Forward(X_test,mask,p) #inputs
                _, predicted = torch.max(outputs, 1)
                test_loss = criterion(outputs, y_test)
                test_acc = 100 * torch.sum(y_test==predicted).double() / len(y_test)
                print('********************{}*************'.format(phase))
                print ('Epoch [%d/%d] Loss: %.4f   Acc: %.4f'%(epoch+1, num_epochs, test_loss.item(), test_acc.item()))
                if test_acc > best_acc:
                    best_acc = test_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

              



    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
          time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    #load best model weights
    model.load_state_dict(best_model_wts)
    return model,losses,accuracies

In [12]:
class Population:   
    #constructor for initialising the population list
    #list of DNA objects
    def __init__(self,m,num,maskLength):
        self.population=[]
        self.mutation_rate=m #mutation rate for mutation
        self.popmax=num #maximum number of entities in the population
        self.maskLength=maskLength
        for i in range (num):
            #creating a dna object
            #an initial random population created 
            dna =DNA(self.maskLength)
            self.population.append (dna)
      
        self.matingPool=[]
    
    #going through all the entities of population 
    #finding fitness of all population entities 
    def calcFitness (self,model):
        
        for i in range(0,self.popmax):
            self.population[i].fitness (model)

    def naturalSelection(self):
        self.matingPool=[]
        maxFitness=0
        for i in range (self.popmax):
            # moving throught the entire population 
            if (self.population[i].fit>maxFitness):
                maxFitness=self.population[i].fit
       
        #max Fitness has the maximum loss score of the entire population  
        #iterating through the all inviduals of the population
        for i in range (self.popmax ):
        
            n=self.Mymap(self.population[i].fit,0,maxFitness,0,1)
        
            n=math.floor(n*100)
            
            for j in range (n):
                self.matingPool.append (self.population[i])#creating mating pool

    def Mymap(self,num,prevlow,prevhigh,nextlow,nexthigh):
        
        prevrange =float((num-prevlow)/(prevhigh-prevlow))
        return nextlow+(nexthigh-nextlow)*prevrange

    def   generate (self):
        for i in range (self.popmax ):
            index_1=math.floor(random.randint  (0,len(self.matingPool)-1))
            index_2=math.floor (random.randint (0,len(self.matingPool)-1))
            parent1=self.matingPool[index_1]
            parent2=self.matingPool[index_2]
            child=parent1.crossover(parent2)
            child.mutate(self.mutation_rate)
            self.population[i]=child
            
    #returns the fiitest individual mask of the population 
    #also returns the keeping probability of the fittest mask 
    def fittest(self):
        fittest=self.population[0]
        for i  in range (self.popmax):
            if (fittest.fit<self.population[i].fit):
                fittest=self.population[i]
        return fittest,fittest.keep_prob()

In [13]:
class DNA: 
    #constructor for the creation of the mask as a gene object 
    def __init__(self,maskLength):
        self.maskLength=maskLength
        self.gene=torch.bernoulli(torch.empty(1,maskLength).uniform_(0,1))#creation of mask 
        self.fit=0

    def keep_prob (self):
        num_one =0
        for i in range (self.maskLength):
            if (self.gene[0,i]==1):
                num_one=num_one+1
        return float(num_one/self.maskLength)
    
    
    #finding the fitness of a particular mask
    #accuracy of all training set is the fitness in one epoch
    #putting model in train mode
    def fitness(self,model):
 
        running_loss=0
        running_corrects=0
        model.train()
        outputs=model.Forward(X_train,self.gene,self.keep_prob())
        _,preds=torch.max(outputs,1)
        loss=criterion (outputs,y_train)
        acc = 100*torch.sum(y_train==torch.max(outputs.data, 1)[1]).double() / len(y_train)
        self.fit=acc
        return acc
        
    #one parent is the passed in the argument 
    #another parent is the one from which this function is called 
    #another parent is self.gene
    def crossover (self,parent2):
        
        child =DNA(self.maskLength)
        midpoint =random .randint (0,self.maskLength-1)
        for i in range (0,self.maskLength):
            if (i>midpoint):
                child.gene [0,i]=self.gene[0,i]
            else :
                child.gene [0,i]=parent2.gene[0,i]
        
        return child
    
    
    #randomly activate some of the nodes  
    #mutate some of the genes
    def mutate(self,mutation_rate):
         
        for i in range (self.maskLength):
            if (random.randint (0,99)<=mutation_rate*100):
                self.gene[0,i]=1 

In [14]:
mutation_rate =0
max_population=30
maskLength=X_train.shape[1]
model=Model(input_size, hidden_size, num_classes)#creating the object of the class
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
#**CONTROL BLOCK** controls the epochs and the generations of mask
#step1: an object of the population class randomly generating the first population 
#step2: calculate fitness of each entitiy of the population 
#step3: creates a mating pool of the population based on the worst two performing parent 
#step4: fittest mask of the generating along with keep_prob found 
#step5: if 0th ,10th ,20th, the epochs starts training on the worst performing mask /other wise new generation is created 

epochgens=0
population =Population(mutation_rate,max_population,maskLength)
total_acc=[]
p=0
while (epochgens<=num_epochs):
    print ('Epoch generations (',epochgens,'/{})'.format(num_epochs),end=' :')
    if p < 0.8:
        population .calcFitness(model)
        population.naturalSelection()
        fittestmask,p = population .fittest()
        accuracy=fittestmask.fitness(model)
        print ("accuracy(fittest mask)",accuracy,"keep_prob",p,end='\n')
    if (epochgens%10==0):
        model,losses,accuracies=train_model(model,criterion,optimizer,fittestmask.gene,p,30)
        total_acc=total_acc+accuracies
    population.generate()
    epochgens+=1

Epoch generations ( 0 /100) :accuracy(fittest mask) tensor(10.9921, dtype=torch.float64) keep_prob 0.4861111111111111
Epoch 0/29
----------
********************train*************
Epoch [1/30] Loss: 2.5472   Acc: 10.9921
********************val*************
Epoch [1/30] Loss: 2.4893   Acc: 6.6667
Epoch 1/29
----------
********************train*************
Epoch [2/30] Loss: 2.4503   Acc: 14.1667
********************val*************
Epoch [2/30] Loss: 2.4435   Acc: 10.0926
Epoch 2/29
----------
********************train*************
Epoch [3/30] Loss: 2.3618   Acc: 19.1667
********************val*************
Epoch [3/30] Loss: 2.3999   Acc: 15.0926
Epoch 3/29
----------
********************train*************
Epoch [4/30] Loss: 2.2812   Acc: 26.0317
********************val*************
Epoch [4/30] Loss: 2.3583   Acc: 19.4444
Epoch 4/29
----------
********************train*************
Epoch [5/30] Loss: 2.2081   Acc: 30.7540
********************val*************
Epoch [5/30] Loss: 2.318

********************train*************
Epoch [10/30] Loss: 1.2948   Acc: 64.5238
********************val*************
Epoch [10/30] Loss: 1.4467   Acc: 57.5926
Epoch 10/29
----------
********************train*************
Epoch [11/30] Loss: 1.2642   Acc: 64.6825
********************val*************
Epoch [11/30] Loss: 1.4205   Acc: 58.3333
Epoch 11/29
----------
********************train*************
Epoch [12/30] Loss: 1.2340   Acc: 65.5159
********************val*************
Epoch [12/30] Loss: 1.3941   Acc: 59.5370
Epoch 12/29
----------
********************train*************
Epoch [13/30] Loss: 1.2050   Acc: 67.1825
********************val*************
Epoch [13/30] Loss: 1.3680   Acc: 60.1852
Epoch 13/29
----------
********************train*************
Epoch [14/30] Loss: 1.1770   Acc: 68.1746
********************val*************
Epoch [14/30] Loss: 1.3427   Acc: 61.1111
Epoch 14/29
----------
********************train*************
Epoch [15/30] Loss: 1.1497   Acc: 68.7698
****

********************val*************
Epoch [18/30] Loss: 0.7882   Acc: 79.5370
Epoch 18/29
----------
********************train*************
Epoch [19/30] Loss: 0.5757   Acc: 86.2302
********************val*************
Epoch [19/30] Loss: 0.7768   Acc: 79.6296
Epoch 19/29
----------
********************train*************
Epoch [20/30] Loss: 0.5635   Acc: 86.6270
********************val*************
Epoch [20/30] Loss: 0.7655   Acc: 79.6296
Epoch 20/29
----------
********************train*************
Epoch [21/30] Loss: 0.5515   Acc: 87.1429
********************val*************
Epoch [21/30] Loss: 0.7543   Acc: 80.1852
Epoch 21/29
----------
********************train*************
Epoch [22/30] Loss: 0.5398   Acc: 87.5000
********************val*************
Epoch [22/30] Loss: 0.7432   Acc: 80.5556
Epoch 22/29
----------
********************train*************
Epoch [23/30] Loss: 0.5283   Acc: 87.6587
********************val*************
Epoch [23/30] Loss: 0.7322   Acc: 80.9259
Epoch 

********************train*************
Epoch [27/30] Loss: 0.3017   Acc: 93.8889
********************val*************
Epoch [27/30] Loss: 0.4765   Acc: 87.8704
Epoch 27/29
----------
********************train*************
Epoch [28/30] Loss: 0.2956   Acc: 93.9683
********************val*************
Epoch [28/30] Loss: 0.4712   Acc: 87.9630
Epoch 28/29
----------
********************train*************
Epoch [29/30] Loss: 0.2898   Acc: 94.2460
********************val*************
Epoch [29/30] Loss: 0.4657   Acc: 88.2407
Epoch 29/29
----------
********************train*************
Epoch [30/30] Loss: 0.2842   Acc: 94.1270
********************val*************
Epoch [30/30] Loss: 0.4603   Acc: 88.4259
Training complete in 0m 1s
Best val Acc: 88.425926
Epoch generations ( 31 /100) :accuracy(fittest mask) tensor(94.0476, dtype=torch.float64) keep_prob 0.6666666666666666
Epoch generations ( 32 /100) :accuracy(fittest mask) tensor(94.1667, dtype=torch.float64) keep_prob 0.6944444444444444
Ep

Epoch generations ( 48 /100) :accuracy(fittest mask) tensor(96.7857, dtype=torch.float64) keep_prob 0.7638888888888888
Epoch generations ( 49 /100) :accuracy(fittest mask) tensor(96.8254, dtype=torch.float64) keep_prob 0.7638888888888888
Epoch generations ( 50 /100) :accuracy(fittest mask) tensor(96.6667, dtype=torch.float64) keep_prob 0.75
Epoch 0/29
----------
********************train*************
Epoch [1/30] Loss: 0.1962   Acc: 96.6667
********************val*************
Epoch [1/30] Loss: 0.3341   Acc: 92.1296
Epoch 1/29
----------
********************train*************
Epoch [2/30] Loss: 0.1918   Acc: 96.6667
********************val*************
Epoch [2/30] Loss: 0.3299   Acc: 92.3148
Epoch 2/29
----------
********************train*************
Epoch [3/30] Loss: 0.1869   Acc: 96.7857
********************val*************
Epoch [3/30] Loss: 0.3259   Acc: 92.1296
Epoch 3/29
----------
********************train*************
Epoch [4/30] Loss: 0.1819   Acc: 96.7857
***************

********************train*************
Epoch [13/30] Loss: 0.0958   Acc: 99.1270
********************val*************
Epoch [13/30] Loss: 0.2102   Acc: 97.0370
Epoch 13/29
----------
********************train*************
Epoch [14/30] Loss: 0.0941   Acc: 99.1270
********************val*************
Epoch [14/30] Loss: 0.2082   Acc: 97.2222
Epoch 14/29
----------
********************train*************
Epoch [15/30] Loss: 0.0924   Acc: 99.2460
********************val*************
Epoch [15/30] Loss: 0.2059   Acc: 97.2222
Epoch 15/29
----------
********************train*************
Epoch [16/30] Loss: 0.0906   Acc: 99.2460
********************val*************
Epoch [16/30] Loss: 0.2032   Acc: 97.2222
Epoch 16/29
----------
********************train*************
Epoch [17/30] Loss: 0.0886   Acc: 99.3254
********************val*************
Epoch [17/30] Loss: 0.2004   Acc: 97.2222
Epoch 17/29
----------
********************train*************
Epoch [18/30] Loss: 0.0868   Acc: 99.2857
****

********************train*************
Epoch [27/30] Loss: 0.0519   Acc: 99.8810
********************val*************
Epoch [27/30] Loss: 0.1364   Acc: 98.8889
Epoch 27/29
----------
********************train*************
Epoch [28/30] Loss: 0.0512   Acc: 99.9206
********************val*************
Epoch [28/30] Loss: 0.1349   Acc: 98.8889
Epoch 28/29
----------
********************train*************
Epoch [29/30] Loss: 0.0505   Acc: 99.9206
********************val*************
Epoch [29/30] Loss: 0.1336   Acc: 98.8889
Epoch 29/29
----------
********************train*************
Epoch [30/30] Loss: 0.0498   Acc: 99.9603
********************val*************
Epoch [30/30] Loss: 0.1323   Acc: 99.0741
Training complete in 0m 1s
Best val Acc: 99.074074
Epoch generations ( 71 /100) :Epoch generations ( 72 /100) :Epoch generations ( 73 /100) :Epoch generations ( 74 /100) :Epoch generations ( 75 /100) :Epoch generations ( 76 /100) :Epoch generations ( 77 /100) :Epoch generations ( 78 /100) :

********************train*************
Epoch [12/30] Loss: 0.0312   Acc: 99.9603
********************val*************
Epoch [12/30] Loss: 0.0990   Acc: 99.5370
Epoch 12/29
----------
********************train*************
Epoch [13/30] Loss: 0.0308   Acc: 99.9603
********************val*************
Epoch [13/30] Loss: 0.0983   Acc: 99.5370
Epoch 13/29
----------
********************train*************
Epoch [14/30] Loss: 0.0304   Acc: 99.9603
********************val*************
Epoch [14/30] Loss: 0.0976   Acc: 99.5370
Epoch 14/29
----------
********************train*************
Epoch [15/30] Loss: 0.0301   Acc: 99.9603
********************val*************
Epoch [15/30] Loss: 0.0969   Acc: 99.5370
Epoch 15/29
----------
********************train*************
Epoch [16/30] Loss: 0.0297   Acc: 99.9603
********************val*************
Epoch [16/30] Loss: 0.0961   Acc: 99.5370
Epoch 16/29
----------
********************train*************
Epoch [17/30] Loss: 0.0293   Acc: 99.9603
****

********************train*************
Epoch [29/30] Loss: 0.0188   Acc: 100.0000
********************val*************
Epoch [29/30] Loss: 0.0724   Acc: 99.7222
Epoch 29/29
----------
********************train*************
Epoch [30/30] Loss: 0.0187   Acc: 100.0000
********************val*************
Epoch [30/30] Loss: 0.0719   Acc: 99.7222
Training complete in 0m 1s
Best val Acc: 99.722222
